**Rust Quantum Library**

Below are dependencies, creates, and global variables.

In [2]:
:dep ndarray = { version = "0.14", features = ["serde"] }
:dep ndarray-linalg = { version = "0.13.1", features = ["intel-mkl-static"]}
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series"] }
:dep thiserror = "1.0"
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep dimensioned = "0.7"
:dep num-complex = "0.4"
:dep qm = { version = "0.1", git ="https://github.com/buff-beacon-project/qm" }

extern crate plotters;
extern crate ndarray;

pub use ndarray_linalg::c64;
use std::f64::consts::PI;
use plotters::prelude::*;
use ndarray::prelude::*;
use ndarray_linalg::*;
use qm::*;

pub type VecC64 = ndarray::Array1<c64>;
pub type MatrixC64 = ndarray::Array2<c64>;

pub const norm_const: f64 = 1./std::f64::consts::SQRT_2;

*The Density Matrix*

The density matrix is a more general way of characterizing the overall state of a quantum system.  It is represented as $\rho = |\psi\rangle\langle\psi|$, where the wavefunction $|\psi\rangle$ is represented in Dirac notation as a column vector, and $\langle\psi|$ is the complex conjugate of the row vector. The density matrix can describe both pure states and mixed states. Pure states represent the state of one particle or the average state of an ensemble. In reality, most of the time mixed states are prepared or show up in nature. For example, light from the sun is a full mixture of quantum states, so it is not unpolarized and considered a maximally mixed state.

Here, we create density matrices out of the four Bell States, which are entangled quantum states:

$$\phi_- = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$$
$$\phi_+ = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$$
$$\psi_- = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$$
$$\psi_+ = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$$

Also, a maximally mixed density matrix can be created by setting all diagonal elements to \frac{1}{d} where d is the dimension of the matrix.

In [3]:
let bell_phi_plus_vec: VecC64 = array![c64::new(norm_const , 0.0) , c64::new(0.0 , 0.0) , c64::new(0.0 , 0.0) , c64::new(norm_const , 0.0) ];

let bell_phi_minus_vec: VecC64 = array![c64::new(norm_const , 0.0) , c64::new(0.0 , 0.0) , c64::new(0.0 , 0.0) , c64::new(-norm_const , 0.0) ];

let bell_psi_plus_vec: VecC64 = array![c64::new(0.0 , 0.0) , c64::new(norm_const , 0.0) , c64::new(norm_const , 0.0) , c64::new(0.0 , 0.0) ];

let bell_psi_minus_vec: VecC64 = array![c64::new(0.0 , 0.0) , c64::new(norm_const , 0.0) , c64::new(-norm_const , 0.0) , c64::new(0.0 , 0.0) ];

let rho_phi_plus: MatrixC64 = create_dens_matrix(&bell_phi_plus_vec);
let rho_phi_minus: MatrixC64 = create_dens_matrix(&bell_phi_minus_vec);
let rho_psi_plus: MatrixC64 = create_dens_matrix(&bell_psi_plus_vec);
let rho_psi_minus: MatrixC64 = create_dens_matrix(&bell_psi_minus_vec);

println!("Density matrix for phi_+ = \n {:.1}", rho_phi_plus );
println!("Density matrix for phi_- = \n {:.1}", rho_phi_minus );
println!("Density matrix for psi_+ = \n {:.1}", rho_psi_plus );
println!("Density matrix for psi_- = \n {:.1}", rho_psi_minus );

let mixed_diag: VecC64 = Array::from_elem( 4, c64::new(0.25, 0.0) );
let rho_mixed: MatrixC64 = MatrixC64::from_diag(&mixed_diag);
println!("Maximally mixed density matrix =\n {:.2}", rho_mixed)

Density matrix for phi_+ = 
 [[0.5+0.0i, 0.0+0.0i, 0.0+0.0i, 0.5+0.0i],
 [0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i],
 [0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i],
 [0.5+0.0i, 0.0+0.0i, 0.0+0.0i, 0.5+0.0i]]
Density matrix for phi_- = 
 [[0.5+0.0i, 0.0+0.0i, 0.0+0.0i, -0.5+0.0i],
 [0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i],
 [0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i],
 [-0.5+0.0i, 0.0+0.0i, 0.0+0.0i, 0.5+0.0i]]
Density matrix for psi_+ = 
 [[0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i],
 [0.0+0.0i, 0.5+0.0i, 0.5+0.0i, 0.0+0.0i],
 [0.0+0.0i, 0.5+0.0i, 0.5+0.0i, 0.0+0.0i],
 [0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i]]
Density matrix for psi_- = 
 [[0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i],
 [0.0+0.0i, 0.5+0.0i, -0.5+0.0i, 0.0+0.0i],
 [0.0+0.0i, -0.5+0.0i, 0.5+0.0i, 0.0+0.0i],
 [0.0+0.0i, 0.0+0.0i, 0.0+0.0i, 0.0+0.0i]]
Maximally mixed density matrix =
 [[0.25+0.00i, 0.00+0.00i, 0.00+0.00i, 0.00+0.00i],
 [0.00+0.00i, 0.25+0.00i, 0.00+0.00i, 0.00+0.00i],
 [0.00+0.00i, 0.00+0.00i, 0.25+0.00i, 0.00+0.00i],
 [0.0

()

*Purity*

Purity is the trace of the square of the density matrix, $Tr(\rho^{2})$. In this example, we continuously add mixtures to a pure Bell state and graph its purity. The purity ranges from $\frac{1}{d}$ to 1, where $d$ is the dimension of the density matrix. In this case with a 4x4 density matrices, the purity ranges from 0.25 to 1.

In [4]:
let mut points: Vec<(f64, f64)> = Vec::new();

for i in 0..100{
    let eta = c64::new((i as f64)/100. , 0.); 
    let mut rho_total = (c64::new(1. , 0.)-eta)*rho_phi_plus.clone() + eta*rho_mixed.clone();
    let mut purity = find_purity(rho_total);
    points.push( (eta.re, purity) );
}

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Purity of partially mixed Bell state", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..1.1, 0_f64..1.01)?;
    
    chart
        .configure_mesh()
        .x_desc("eta")
        .y_desc("Purity")
        .draw()?;
    chart.draw_series(points.into_iter().map(|(x,y)| Circle::new((x,y), 3, BLUE.filled())));
    Ok(())
});

figure

Purity of partially mixed Bell state
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Purity
 
 
eta
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
1.0

*Concurrence*

Here we graph concurrence, a measure of entanglement between two qubits. The calculation essentially tests how multiplying the Pauli-y spin matrix to one of the qubits effects the other qubit. On the x axis is $\theta$, and on the y axis is the concurrence of the pure density matrix $\rho_{pure}$ created from the partially entangled state $\psi = \sin\theta|00\rangle + \cos\theta|11\rangle$.

In [5]:
let mut points_x: Vec<f64> = Vec::new();
let mut points_y: Vec<f64> = Vec::new();

for i in 0..91{
    let mut theta: f64 = (i as f64)*PI/180.;
    let mut psi_part_entangled: VecC64 = array![c64::new(theta.cos(), 0.0), c64::new(0.0, 0.0), c64::new(0.0, 0.0), c64::new(theta.sin(), 0.0)];
    let mut rho_part_entangled: MatrixC64 = create_dens_matrix(&psi_part_entangled);
    let mut concurrence: f64 = find_concurrence(rho_part_entangled);
    points_x.push(theta*180./PI);
    points_y.push(concurrence);
}

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Concurrence of partially entangled state as a function of theta", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..90_f64, 0_f64..1_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("theta")
        .y_desc("Concurrence")
        .draw()?;
    
    chart.draw_series(LineSeries::new(
        points_x.into_iter().zip(points_y.into_iter()), &RED,))?;

    Ok(())
});
figure

eigvals = [0.0, 0.0, 0.0, 0.0]
eigval sum = 0
eigvals = [-0.0000000000000000000016940658945086007, 0.0, 0.0, 0.03489949670250097]
eigval sum = -0.0000000000000000000016940658945086007
eigvals = [0.0, 0.0, 0.000000000029103830443181176, 0.0697564737441253]
eigval sum = 0.000000000029103830443181176
eigvals = [0.0, 0.0, 0.00000000000000000005421010862427522, 0.10452846326765344]
eigval sum = 0.00000000000000000005421010862427522
eigvals = [-0.00000000000000000010842021724855044, 0.0, 0.0, 0.1391731009600654]
eigval sum = -0.00000000000000000010842021724855044
eigvals = [-0.00000000000000000017142388239197287, 0.0, 0.0000000001669911491610208, 0.17364817766693033]
eigval sum = 0.00000000016699114898959693
eigvals = [-0.0000000000000000013991732416483032, 0.0, 0.0000000000000000005318115036598994, 0.2079116908177592]
eigval sum = -0.000000000000000000867361737988404
eigvals = [-0.0000000000000000021895750033251852, 0.0, 0.000000000525505871906834, 0.24192189559966762]
eigval sum = 0.000000

Concurrence of partially entangled state as a function of theta
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Concurrence
 
 
theta
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="55,344 61,333 67,322 73,311 79,300 85,289 91,278 97,268 103,257 109,246 115,236 121,226 127,216 133,206 139,196 145,187 151,177 157,168 163,159 169,150 175,142 181,133 187,125 193,118 199,110 205,103 211,96 217,89 223,83 229,77 235,72 241,66 247,61 253,57 259,52 265,48 271,45 277,42 283,39 289,36 295,34 301,33 307,31 313,31 319,31 325,31 331,31 337,31 343,31 349,33 355,34 361,36 367,39 373,42 379,45 385,48 391,52 397,57 403,61 409,66 415,72 421,77 427,83 433,89 439,96 445,103 451,110 457,118 463,125 469,133 475,142 481,150 487,159 493,168 499,177 505,187 511,196 517,206 523,216 529,226 535,236 541,246 547,257 553,268 559,278 565,289 571,300 577,311 583,322 589,333 595,344 "/>

*Concurrence - adding in a mixed state*

In this example, the 2D rectangular plot shows $\theta$ on the x axis and $\eta$ on the y axis. The concurrence is plotted as a function of $\eta\rho_{pure} + (1-\eta)\rho_{mixed}$. The pure density matrix is the density matrix of the partially entangled state $\psi = \sin\theta|00\rangle + \cos\theta|11\rangle$, and $\rho_{mixed} = 0.25I$ where $I$ is the 4x4 idenity matrix.

In [ ]:
let mut points: Vec<(f64, f64, f64)> = Vec::new();

for eta_index in 0..100{
    let mut eta = c64::new((eta_index as f64)/100. , 0.);
  
    for theta_index in 0..360{
        let mut theta: f64 = (theta_index as f64)*PI/180.;
        let mut psi_part_entangled: VecC64 = array![c64::new(theta.cos(), 0.0), c64::new(0.0, 0.0), c64::new(0.0, 0.0), c64::new(theta.sin(), 0.0)];
        let mut rho_pure: MatrixC64 = create_dens_matrix(&psi_part_entangled);
 
        let mut rho_total = eta*rho_pure.clone() + (1.-eta)*rho_mixed.clone();
        let mut concurrence: f64 = find_concurrence(rho_total);
        points.push( (theta, eta.re, concurrence) ); 
    }

}


let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Concurrence of eta*rho_pure + (1.-eta)*rho_mixed", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..2.*PI, 0_f64..1_f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_desc("theta")
        .y_desc("eta")
        .draw()?;
    
    chart.draw_series(
        points.into_iter()
            .map(|(theta, eta, concurrence)| {
                Rectangle::new(
                    [ (theta, eta) , (theta + PI/18., eta + 0.01) ], 
                    HSLColor(
                        240.0 / 360.0 - 240.0 / 360.0 * (concurrence),
                        0.7,
                        0.1 + 0.4 * concurrence,
                    ).filled(),
                )
            })
    );
    
    
    Ok(())
});

figure

eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum = 0.75
eigvals = [0.25, 0.25, 0.25, 0.25]
eigval sum 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



eigvals = [0.23499999999999993, 0.23500000000000007, 0.24328009900447434, 0.2849596012320143]
eigval sum = 0.7132800990044743
eigvals = [0.235, 0.23500000000000004, 0.2425991102010449, 0.28575949822136404]
eigval sum = 0.7125991102010449
eigvals = [0.23499999999999996, 0.23500000000000007, 0.24194425538423783, 0.2865329449128816]
eigval sum = 0.7119442553842379
eigvals = [0.23499999999999993, 0.23500000000000004, 0.24131611595622976, 0.28727878254336847]
eigval sum = 0.7113161159562298
eigvals = [0.2349999999999999, 0.23499999999999996, 0.24071524259332733, 0.2879958878097307]
eigval sum = 0.7107152425933272
eigvals = [0.23499999999999993, 0.23499999999999993, 0.2401421556132585, 0.2886831752757553]
eigval sum = 0.7101421556132583
eigvals = [0.23499999999999988, 0.235, 0.23959734536220753, 0.28933959971551004]
eigval sum = 0.7095973453622074
eigvals = [0.235, 0.2350000000000001, 0.2390812726163679, 0.2899641583857534]


*Fidelity*

This function is a distance measurement of two density matrices $\rho_{1}$ and $\rho_{2}$. It is expressed as $F=tr\sqrt{\sqrt{\rho_{1}}\rho_{2}\sqrt{\rho_{1}}}$. Here we graph a Fidelity over eta, where the pure state $\rho_{pure}$ stays constant but $\eta$ and $\theta$ are varied. The pure state, $\rho_{1}$, is the density matrix of the Bell State $\psi_- = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$. The mixed density matrix is $\rho_{2}$ = $\eta\rho_{pure} + (1-\eta)\rho_{mixed}$. 

In [ ]:
let mut points: Vec<(f64, f64)> = Vec::new();

for i in 0..100{
    let eta = c64::new((i as f64)/100. , 0.); 
    let mut rho_total = eta*rho_psi_minus.clone() + (c64::new(1. , 0.) - eta)*rho_mixed.clone();
    let mut fidelity = find_fidelity(rho_psi_minus.clone(),rho_total);
    points.push( (eta.re, fidelity) );
}

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Fidelity of rho_1 and rho_2", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..1_f64, 0_f64..1_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("eta")
        .y_desc("Fidelity")
        .draw()?;
    chart.draw_series(points.into_iter().map(|(x,y)| Circle::new((x,y), 3, BLUE.filled())));
    Ok(())
});

figure

*Negativity*

The negativity is expressed as $\frac{||\rho^{\Gamma_{A}}||_{1}-1}{2}$, and it prodives a measure of entanglement between states. It ranges from 0 to $\frac{2}{d}$, where $d$ is the dimension of the matrix. $||\rho^{\Gamma_{A}}||_{1}$ is the trace norm of the partial transpose of $\rho$. 

In [ ]:
let mut points: Vec<(f64, f64, f64)> = Vec::new();

for eta_index in 0..100{
    let mut eta = c64::new((eta_index as f64)/100. , 0.);
  
    for theta_index in 0..360{
        let mut theta: f64 = (theta_index as f64)*PI/180.;
        let mut psi_part_entangled: VecC64 = array![c64::new(theta.cos(), 0.0), c64::new(0.0, 0.0), c64::new(0.0, 0.0), c64::new(theta.sin(), 0.0)];
        let mut rho_pure: MatrixC64 = create_dens_matrix(&psi_part_entangled);
 
        let mut rho_mixed = eta*rho_pure.clone() + (1. - eta)*rho_mixed.clone();
        let mut negativity: f64 = find_negativity(rho_mixed);
        points.push( (theta, eta.re, negativity) ); 
    }

}


let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Negativity of mixed state and partially entangled state", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..2.*PI, 0_f64..1_f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_desc("theta")
        .y_desc("eta")
        .draw()?;
    
    chart.draw_series(
        points.into_iter()
            .map(|(theta, eta, negativity)| {
                Rectangle::new(
                    [ (theta, eta) , (theta + PI/18., eta + 0.01) ], 
                    HSLColor(
                        240.0 / 360.0 - 240.0 / 360.0 * (negativity),
                        0.7,
                        0.5 + negativity,
                    ).filled(),
                )
            })
    );
    
    
    Ok(())
});

figure

In [ ]:
*Trace Norm*

In [ ]:
let mut points: Vec<(f64, f64)> = Vec::new();

for i in 0..100{
    let eta = c64::new((i as f64)/100. , 0.); 
    let mut rho_total = eta*rho_psi_minus.clone() + (c64::new(1. , 0.) - eta)*rho_mixed.clone();
    let mut trace_norm = find_trace_norm(rho_total);
    points.push( (eta.re, trace_norm) );
}

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Trace norm of eta*rho_psi_minus + (1-eta)*rho_mixed", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0_f64..1f64, -1_f64..4_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("eta")
        .y_desc("Trace Norm")
        .draw()?;
    chart.draw_series(points.into_iter().map(|(x,y)| Circle::new((x,y), 3, BLUE.filled())));
    Ok(())
});

figure